# File generator with particles' coordinates for each video

In [1]:
%reset
#Imports
import matplotlib as mpl
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
import scipy.optimize as spo

from tqdm import tqdm_notebook

idx = pd.IndexSlice
%matplotlib inline

from scipy.interpolate import make_interp_spline, BSpline

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
#Function to establish the centre of a circle inthe origin. 
import scipy.optimize as spo
def get_center(trj):
    
    def calc_R(xc, yc):
        """ calculate the distance of each 2D points from the center (xc, yc) """
        return np.sqrt((trj.x.values-xc)**2 + (trj.y.values-yc)**2)

    def f_2(c):
        """ calculate the algebraic distance between the data points and the mean circle centered at c=(xc, yc) """
        Ri = calc_R(*c)
        return Ri - Ri.mean()    

    center_estimate = 0, 0
    center, ir = spo.leastsq(f_2, center_estimate)
    return center

In [3]:
def changeToPolarCoordinates(t):
    
    #Obtention of the circle centre defined by the trajectory for each particle 
    center = get_center(t)
    t["x0"] = t.x-center[0]
    t["y0"] = t.y-center[1]

    #New calculation of the ro and theta coordinates with the coordinates origin correction 
    t["r"] = np.sqrt(t.x0**2 + t.y0**2)
    t["theta"] = np.arctan2(t.y0,t.x0)
    
    return t

In [4]:
def movementAnalyzer(t, timestamp, omega):
    
    t["theta_prime"] = np.mod(np.unwrap(t.theta)-omega*timestamp,2*np.pi) 
    t["theta_prime_unwrap"] = np.unwrap(t.theta_prime)
    t["d_theta_prime"] = t.theta_prime_unwrap.diff()
    
    t["theta_dot_prime"] = t.d_theta_prime/timestamp.diff()
    
    #radius = t.r.mean()*0.1805 #0.1805 um/px

    #t["arc_displace"] = t.d_theta_prime*radius
    #t["arc_displace_prime"] = t.theta_dot_prime.diff()*radius
    
    return t

In [6]:
#Loading the tracking matrix from csv file 
filePath = "Tracking_Test6_20190611.dat" #"Tracking_Test6_20190122.dat"
t = pd.read_csv(filePath ,sep = "\t", index_col=[0])
t = t.set_index(["frame","particle"])
t = t.filter(["x","y"])
t = t.sort_index()
t.head()

,,x,y
frame,particle,,
0,0,353.105185,273.522246
1,0,353.485737,272.946524
2,0,350.367466,271.535337
3,0,350.562027,271.283477
4,0,350.548751,271.296038


In [7]:
#Loading the timestamps of the video to have the time of each position
tmstmp_filePath = r"C:\Users\Eric\Desktop\BASEP_tests\20190611\Test6_2019_06_11_16_09_19.dat"
timestamp = pd.read_csv(tmstmp_filePath, sep="\r", header = None,names=["time"])

#Calculation of timestamps in nanoseconds
timestamp["datetime"] =  pd.to_datetime(timestamp.time+2*3600, unit="s", origin=pd.Timestamp('1904-01-01'))
timestamp["ellapsed_time"] = (timestamp.datetime-timestamp.datetime[0])
timestamp["time"] = timestamp.ellapsed_time.dt.total_seconds()
timestamp.head()

,time,datetime,ellapsed_time
0,0.00,2019-06-13 15:31:01.361999989,00:00:00
1,0.01,2019-06-13 15:31:01.372000217,00:00:00.010000
2,0.40,2019-06-13 15:31:01.762000084,00:00:00.400000
3,0.41,2019-06-13 15:31:01.772000313,00:00:00.410000
4,0.42,2019-06-13 15:31:01.782000065,00:00:00.420000


In [8]:
#*****************************************************
#*****************************************************
#************Change ang vel value*********************
#*****************************************************
#*****************************************************
omega = 0.

t = changeToPolarCoordinates(t)
t["time"] = np.NaN

for p, t_sub in tqdm_notebook(t.groupby("particle")):
    
    particle_i = t.loc[t.index.get_level_values("particle") == p].reset_index(drop=True)
    
    particle_i = changeToPolarCoordinates(particle_i)
    particle_i = movementAnalyzer(particle_i, timestamp.time[:len(particle_i)], omega)
    
    particle_i.index = t.loc[idx[:,p],:].index

    t.loc[idx[:,p],"theta_prime"] = particle_i.theta_prime
    #t.loc[idx[:,p],"theta_prime_unwrap"] = particle_i.theta_prime_unwrap
    #t.loc[idx[:,p],"d_theta_prime"] = particle_i.d_theta_prime
    t.loc[idx[:,p],"theta_dot_prime"] = particle_i.theta_dot_prime
    t.loc[idx[:,p],"time"] = timestamp.time[:len(particle_i)].tolist()



In [9]:
#t = t.filter(["r","time","theta_prime", "theta_dot_prime"])
t

,,x,y,x0,y0,r,theta,time,theta_prime,theta_dot_prime
frame,particle,,,,,,,,,
0,0,353.105185,273.522246,416967.373720,328953.431414,531104.651445,0.667948,0.000,0.667948,NaN
1,0,353.485737,272.946524,416967.754272,328952.855692,531104.593626,0.667946,0.010,0.667946,-1.294821e-04
2,0,350.367466,271.535337,416964.636001,328951.444505,531101.271433,0.667948,0.400,0.667948,3.975594e-06
3,0,350.562027,271.283477,416964.830563,328951.192645,531101.268186,0.667947,0.410,0.667947,-5.991946e-05
4,0,350.548751,271.296038,416964.817287,328951.205206,531101.265543,0.667947,0.420,0.667947,3.405134e-06
5,0,350.789613,271.583011,416965.058148,328951.492179,531101.632386,0.667947,0.430,0.667947,1.433157e-05
6,0,350.877504,271.714763,416965.146040,328951.623931,531101.782994,0.667947,0.440,0.667947,9.226319e-06
7,0,351.166126,271.621636,416965.434662,328951.530804,531101.951908,0.667947,0.450,0.667947,-4.742460e-05
8,0,351.344725,272.044289,416965.613261,328951.953458,531102.353907,0.667947,0.460,0.667947,4.165085e-05


# SAVE

In [20]:
#Save to .csv file
t.to_csv("Coordinates_Test6_20190611.dat", sep = "\t")